# Neural Networks (2025-01):  Homework 1
(Due: 2025.4.29 11:59 pm)

**Formatting: please include both a .ipynb and .pdf file in your homework submission, named studentID_homework1.ipynb and studentID_homework1.pdf.  Please don't change the filename.**       


## [Prerequisite] Development Evironment Setup

Note that this notebook is written in CUDA V12.5 and Python 3.11. We will usually use Pytorch and torchivison for out NN assignments. You may have compile error if the version of python libraries is mismatched. Check the version of packages you have installed. Google Colab has CUDA pre-installed and make sure the installed CUDA version.

In [1]:
!nvcc --version
!python -V
!pip show torch torchvision

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Python 3.11.12
Name: torch
Version: 2.4.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, nvidia-cublas-cu12, nvidia-cuda-cupti-cu12, nvidia-cuda-nvrtc-cu12, nvidia-cuda-runtime-cu12, nvidia-cudnn-cu12, nvidia-cufft-cu12, nvidia-curand-cu12, nvidia-cusolver-cu12, nvidia-cusparse-cu12, nvidia-nccl-cu12, nvidia-nvtx-cu12, sympy, triton, typing-extensions
Required-by: accelerate, fastai, pandasai, peft, sentence-transformers, timm, torchaudio, torchvision
---
Name: torchvision
Version: 0.19.0
Summary: image and video datasets and models for torch deep le

In my case, there were version mismatches between torch, python and torchvision. For stable version versions of torch and torchvision, see [here](https://pypi.org/project/torchvision/). You may upgrade or downgrade one or more libraries to properly start your project. For example, I needed to downgrade the version of torchvision to v0.19.

In [12]:
!pip install torchvision==0.19.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.3/797.3 MB 1.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1
    Uninstalling torch-2.4.1:
      Successfully uninstalled torch-2.4.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled torchvision-0.21.0+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandasai 2.4.2 requires torch==2.4.1; sys_platform != "darwin", but you have torch 2.4.0 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.4.0 which is incompatible.


Check your gpu is properly working. If you are given False after runnig the cell below, change your runtime to gpu (Go to Menu > Runtime > Change runtime).

In [2]:
import torch
torch.cuda.is_available()

True

# 1. Logistic Regression for Binary Classification

In multiclass classification we typically use the: exponential model

$$p(y|\mathbf{o}) = \mathrm{softmax}(\mathbf{o})_y = \frac{\exp(o_y)}{\sum_{y'} \exp(o_{y'})}$$

1.1. For binary classification, i.e. whenever we have only two classes $\{-1, 1\}$, we can arbitrarily set $o_{-1} = 0$. Using the shorthand $o = o_1$ show that this is equivalent to

$$p(y=1|o) = \frac{1}{1 + \exp(-o)}$$

1.2. Show that the log-likelihood loss (often called logistic loss) for labels $y \in \{-1, 1\}$ is thus given by

$$-\log p(y|o) = \log (1 + \exp(-y \cdot o))$$


Now, we will model covariate shift and attempt to fix it using logistic regression. This is a fairly realistic scenario for data scientists. To keep things well under control and understandable we will use [Fashion-MNIST](http://d2l.ai/chapter_linear-networks/fashion-mnist.html) as the data to experiment on.

Follow the instructions from the Fashion MNIST notebook to get the data.

In [9]:
%matplotlib inline
import time
import numpy as np
import matplotlib.pyplot as plt

# Pre-defined libraries in Pytorch
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.init
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn

mnist_train = datasets.FashionMNIST(root='MNIST_data/',train=True, download=True)
mnist_test = datasets.FashionMNIST(root='MNIST_data/',train=False, download=True)

## 2. Logistic Regression

1. Implement the logistic loss function $l(y,f) = -\log(1 + \exp(-y f))$.

In [ ]:
def loss(y,o):
    ## add your loss function here
    return l

2. Plot its values and its derivative for $y = 1$ and $f \in [-5, 5]$, using automatic differentiation.
3. Generate training and test datasets for a binary classification problem using Fashion-MNIST with class $1$ being a combination of `sneaker` and `pullover` and class $-1$ being the combination of `sandal` and `shirt` categories.
3. Train a binary classifier of your choice (it can be linear or a simple MLP such as from a previous lecture) using half the data (i.e. $12,000$ observations mixed as abvove) and one using the full dataset (i.e. $24,000$ observations as arising from the 4 categories) and report its accuracy.

Hint - you should encapsulate the training and reporting code in a callable function since you'll need it quite a bit in the following.

## 3. Covariate Shift
Your goal is to introduce covariate shit in the data and observe the accuracy. For this, compose a dataset of  12,000  observations, given by a mixture of sneaker and pullover and of sandal and shirt respectively, where you use a fraction  λ∈{0.05,0.1,0.2,…0.8,0.9,0.95}  of one and a fraction of  1−λ  of the other datasets respectively. For instance, you might pick for  λ=0.1  a total of  600  sneaker and  5,400  pullover images and likewise  600  sandal and  5,400  shirt photos, yielding a total of  12,000  images for training. Note that the test set remains unbiased, composed of  2,000  photos for the sneaker + pullover category and of the sandal + shirt category each.

Generate training sets that are appropriately biased. You should have 11 datasets.
Train a binary classifier using this and report the test set accuracy on the unbiased test set.

In [ ]:
    ## add your code here

## 4. Covariate Shift Correction

Having observed that covariate shift can be harmful, let's try fixing it. For this we first need to compute the appropriate propensity scores $\frac{dp(x)}{dq(x)}$. For this purpose pick a biased dataset, let's say with $\lambda = 0.1$ and try to fix the covariate shift.

1. When training a logistic regression binary classifier to fix covariate shift, we assumed so far that both sets are of equal size. Show that re-weighting data in training and test set appropriately can help address the issue when both datasets have different size. What is the weighting?
2. Train a binary classifier (using logistic regression) distinguishing between the biased training set and the unbiased test set. Note - you need to weigh the data.
3. Use the scores to compute weights on the training set. Do they match the weight arising from the biasing distribution $\lambda$?
4. Train a binary classifier of the covariate shifted problem using the weights obtained previously and report the accuracy. Note - you will need to modify the training loop slightly such that you can compute the gradient of a weighted sum of losses.


In [ ]:
    ## add your loss function here
